# Credit Default Swaps (CDS) Fundamentals

This notebook demonstrates how to access and use the **Credit Default Swap (CDS)** functionalities within the **LSEG Financial Analytics SDK**. 

**Credit Default Swaps** are financial derivatives that allow investors to transfer or hedge credit risk. They provide protection against default events of reference entities (corporations, governments, etc.), making them essential tools for credit risk management and structured credit products.

**You will be able to:**
- Define **Single Name CDS** instruments with **instrument code (RIC)**
- Configure pricing parameters
- Evaluate CDS valuation analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.cds` - for CDS instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import cds

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a CDS instrument you need to follow a structured 3-step process:

1. **CDS Definition** - Specify basic CDS with instrument code (RIC)
2. **CDS Instrument Definition** - Create the instrument object
3. **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# Create CDS definition with instrument code (RIC)
cds_definition = cds.CdsDefinition(
    instrument_code="BNPP5YEUAM=R"                                          # BNP Paribas 5Y EUR CDS RIC
)

# Create CDS instrument from definition
cds_instrument = cds.CdsDefinitionInstrument(
    definition=cds_definition
)

print("CDS instrument created from RIC")

# Configure pricing parameters, optional
pricing_params = cds.CdsPricingParameters(
    valuation_date = dt.datetime.strptime("2023-03-01", "%Y-%m-%d"),          # Valuation date
)
print("Pricing parameters configured")

CDS instrument created from RIC
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function with error handling
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = cds.price(
        definitions=[cds_instrument],
        pricing_preferences=pricing_params,
        fields="accruedAmountInDealCcy, upfrontAmountInDealCcy, MarketValueInDealCcy, cleanMarketValueInDealCcy"  # optional
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("CDS pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

CDS pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g. instrument code, reference entity, dates, notional, spread), it's CdsDefinition that we used. 

 - **pricingPreferences**: Valuation date, price side, currency settings.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **description**: Instrument summary CdsDefinition and also the default fields not specified in the CdsDefinition, but used by default in the calculation
   - **valuation**: customizable list of analytics (`marketValueInDealCcy`, `spreadBp`, `upfrontAmount`)
   - **error**: root cause of the issue, if any

In [4]:
 # Access the valuation object
valuation = response.data.analytics[0].valuation
print(json.dumps(valuation.as_dict(), indent = 4))

{
    "accruedAmountInDealCcy": -2000.0,
    "upfrontAmountInDealCcy": -22251.8900987992,
    "marketValueInDealCcy": -24243.8352187997,
    "cleanMarketValueInDealCcy": -22244.4994870251
}


In [5]:
# Convert to DataFrame
df_valuation = pd.DataFrame(list(valuation.items()), columns=["Fields", "Value"])
# Display the DataFrame
display(df_valuation)

,Fields,Value
0,accruedAmountInDealCcy,-2000.000000
1,upfrontAmountInDealCcy,-22251.890099
2,marketValueInDealCcy,-24243.835219
3,cleanMarketValueInDealCcy,-22244.499487


## Appendix: Complete Functions Reference

The following is a complete list of all classes, enums, and functions available after importing from `lseg_analytics.pricing.instruments.cds`:

**Core Classes:**
- `CdsDefinition` - Define CDS instrument specifications
- `CdsDefinitionInstrument` - Instrument container for pricing operations
- `CdsPricingParameters` - Configure pricing parameters and market data settings
- `PremiumLegDefinition` - Premium leg configuration
- `ProtectionLegDefinition` - Protection leg configuration

**Response Classes:**
- `CdsCalculationResponse` - Main response container
- `CdsAnalyticsResponseWithError` - Response with error
- `CdsDescription` - Description of the instrument
- `CdsValuation` - Individual instrument valuation results
- `CdsAnalyticsResponseData` - Detailed analytics and calculations
- `CdsCashflows` - Cashflow information
- `CdsNominalMeasures` - Nominal risk measures
- `CdsSpreadMeasures` - Spread-based measures
- `CdsPricingAnalysis` - Pricing analysis

**Configuration Enums:**
- `IPADirectionEnum` - Buy/Sell protection direction (PAID, RECEIVED)
- `CdsConventionEnum` - CDS conventions (ISDA, USER_DEFINED)
- `InterestPaymentFrequencyEnum` - Payment frequencies (QUARTERLY, SEMIANNUALLY, etc.)
- `InterestCalculationMethodEnum` - Interest calculation method (DCB_30_360, DCB_ACTUAL_360, etc.)
- `SeniorityEnum` - Debt seniority levels (SENIOR_UNSECURED, SUBORDINATED, etc.)
- `DocClauseEnum` - Documentation clauses (MODIFIED_RESTRUCT03, CUM_RESTRUCT14, etc.)
- `AccruedCalculationMethodEnum` - Accrued interest calculation methods (DCB_30_360, DCB_ACTUAL_360, etc.)
- `PaymentBusinessDayConventionEnum`, `StartDateMovingConventionEnum` and `EndDateMovingConventionEnum` - Payment dates/start date/end date business day conventions (MODIFIED_FOLLOWING, NEXT_BUSINESS_DAY, etc.)
- `FinancialContractStubRuleEnum` - Defines whether coupon roll dates are calculated backwards from maturity or forward from the start date (ISSUE, LONG_FIRST_FULL, MATURITY, etc.)

**Market Data Classes:**
- `MarketDataQps` - Market data configurations

**Main Functions:**
- `price()` - Execute CDS pricing calculations